In [6]:
%reload_ext autoreload
%autoreload 2

In [7]:
import os, sys
import re
import json
import glob
import datetime
from collections import Counter

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from wordcloud import WordCloud

In [8]:
# Add parent directory to path to import modules from src

rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import SlackDataLoader
from src.loader import slack_parser
import src.utils as utils

In [9]:

data = SlackDataLoader("../anonymized")


In [10]:
#get users

user = data.get_users()
df = pd.DataFrame(user)
print(type(user))


<class 'list'>


In [27]:
# a channel
channels = data.get_channels()
df = pd.DataFrame(channels)
df.head(20)

,id,name,created,creator,is_archived,is_general,members,topic,purpose,pins
0,C03T0APHX63,all-community-building,1660301317,U03TEPYRM2P,False,False,"[U03T89ACUUW, U03TEPYRM2P, U03TNP8Q8CT, U03TT5...","{'value': '', 'creator': '', 'last_set': 0}","{'value': '', 'creator': '', 'last_set': 0}",NaN
1,C03T0AX4K6K,all-technical-support,1660301462,U03TEPYRM2P,False,False,"[U03T89ACUUW, U03TEPYRM2P, U03TNP8Q8CT, U03TT5...","{'value': '', 'creator': '', 'last_set': 0}","{'value': '', 'creator': '', 'last_set': 0}",NaN
2,C03T89KDGA2,all-career-exercises,1660301361,U03TEPYRM2P,False,False,"[U03T89ACUUW, U03TEPYRM2P, U03TNP8Q8CT, U03TT5...","{'value': '', 'creator': '', 'last_set': 0}","{'value': '', 'creator': '', 'last_set': 0}","[{'id': '1663839365.770289', 'type': 'C', 'cre..."
3,C03T89PMJKG,all-resources,1660301441,U03TEPYRM2P,False,False,"[U03T89ACUUW, U03TEPYRM2P, U03TNP8Q8CT, U03TT5...","{'value': '', 'creator': '', 'last_set': 0}","{'value': '', 'creator': '', 'last_set': 0}","[{'id': '1667584331.857749', 'type': 'C', 'cre..."
4,C03TBUCU4UD,random,1660300985,U03TEPYRM2P,False,False,"[U03T89ACUUW, U03TEPYRM2P, U03TNP8Q8CT, U03TT5...","{'value': '', 'creator': '', 'last_set': 0}","{'value': 'This channel is for... well, everyt...",NaN
5,C03TEQM38HH,all-ideas,1660301408,U03TEPYRM2P,False,False,"[U03T89ACUUW, U03TEPYRM2P, U03TNP8Q8CT, U03TT5...","{'value': '', 'creator': '', 'last_set': 0}","{'value': '', 'creator': '', 'last_set': 0}",NaN
6,C03TEQQS9NF,all-week1,1660301480,U03TEPYRM2P,False,False,"[U03T89ACUUW, U03TEPYRM2P, U03TNP8Q8CT, U03TT5...","{'value': '', 'creator': '', 'last_set': 0}","{'value': '', 'creator': '', 'last_set': 0}","[{'id': '1661638084.234439', 'type': 'C', 'cre..."
7,C03U4J8J4LQ,all-broadcast,1660300985,U03TEPYRM2P,False,True,"[U03T89ACUUW, U03TEPYRM2P, U03TNP8Q8CT, U03TT5...","{'value': '', 'creator': '', 'last_set': 0}",{'value': 'This is the one channel that will a...,NaN
8,C03UG4LHM8A,tenx-bot,1661016491,U03UUN8M4RX,False,False,[U03UUN8M4RX],"{'value': '', 'creator': '', 'last_set': 0}","{'value': '', 'creator': '', 'last_set': 0}",NaN
9,C03V3LK61QX,team-10,1661768995,U03UJGRN5E0,False,False,"[U03UG0SFHGT, U03UH397319, U03UJGRN5E0]","{'value': 'Week 2 challenge-A/B testing', 'cre...","{'value': '', 'creator': '', 'last_set': 0}",NaN


In [26]:
# messages in a file
messages = slack_parser(data.path+"/all-week6")
messages_df = pd.DataFrame(messages)
df.head(20)

../anonymized/all-week6


,id,name,created,creator,is_archived,is_general,members,topic,purpose,pins
0,C03T0APHX63,all-community-building,1660301317,U03TEPYRM2P,False,False,"[U03T89ACUUW, U03TEPYRM2P, U03TNP8Q8CT, U03TT5...","{'value': '', 'creator': '', 'last_set': 0}","{'value': '', 'creator': '', 'last_set': 0}",NaN
1,C03T0AX4K6K,all-technical-support,1660301462,U03TEPYRM2P,False,False,"[U03T89ACUUW, U03TEPYRM2P, U03TNP8Q8CT, U03TT5...","{'value': '', 'creator': '', 'last_set': 0}","{'value': '', 'creator': '', 'last_set': 0}",NaN
2,C03T89KDGA2,all-career-exercises,1660301361,U03TEPYRM2P,False,False,"[U03T89ACUUW, U03TEPYRM2P, U03TNP8Q8CT, U03TT5...","{'value': '', 'creator': '', 'last_set': 0}","{'value': '', 'creator': '', 'last_set': 0}","[{'id': '1663839365.770289', 'type': 'C', 'cre..."
3,C03T89PMJKG,all-resources,1660301441,U03TEPYRM2P,False,False,"[U03T89ACUUW, U03TEPYRM2P, U03TNP8Q8CT, U03TT5...","{'value': '', 'creator': '', 'last_set': 0}","{'value': '', 'creator': '', 'last_set': 0}","[{'id': '1667584331.857749', 'type': 'C', 'cre..."
4,C03TBUCU4UD,random,1660300985,U03TEPYRM2P,False,False,"[U03T89ACUUW, U03TEPYRM2P, U03TNP8Q8CT, U03TT5...","{'value': '', 'creator': '', 'last_set': 0}","{'value': 'This channel is for... well, everyt...",NaN
5,C03TEQM38HH,all-ideas,1660301408,U03TEPYRM2P,False,False,"[U03T89ACUUW, U03TEPYRM2P, U03TNP8Q8CT, U03TT5...","{'value': '', 'creator': '', 'last_set': 0}","{'value': '', 'creator': '', 'last_set': 0}",NaN
6,C03TEQQS9NF,all-week1,1660301480,U03TEPYRM2P,False,False,"[U03T89ACUUW, U03TEPYRM2P, U03TNP8Q8CT, U03TT5...","{'value': '', 'creator': '', 'last_set': 0}","{'value': '', 'creator': '', 'last_set': 0}","[{'id': '1661638084.234439', 'type': 'C', 'cre..."
7,C03U4J8J4LQ,all-broadcast,1660300985,U03TEPYRM2P,False,True,"[U03T89ACUUW, U03TEPYRM2P, U03TNP8Q8CT, U03TT5...","{'value': '', 'creator': '', 'last_set': 0}",{'value': 'This is the one channel that will a...,NaN
8,C03UG4LHM8A,tenx-bot,1661016491,U03UUN8M4RX,False,False,[U03UUN8M4RX],"{'value': '', 'creator': '', 'last_set': 0}","{'value': '', 'creator': '', 'last_set': 0}",NaN
9,C03V3LK61QX,team-10,1661768995,U03UJGRN5E0,False,False,"[U03UG0SFHGT, U03UH397319, U03UJGRN5E0]","{'value': 'Week 2 challenge-A/B testing', 'cre...","{'value': '', 'creator': '', 'last_set': 0}",NaN



### Columns we can get from a slack message<br>

message_type, message_content, sender_id, time_sent, message_distribution, time_thread_start, reply_count, reply_user_count, time_thread_end, reply_users

From a single slack message, we can get <br>

1. The message<br>
2. Type (message, file, link, etc)<br>
3. The sender_id (assigned by slack)<br>
4. The time the message was sent<br>
5. The team (i don't know what that is now)<br>
6. The type of the message (broadcast message, inhouse, just messgae)<br>
7. The thread the message generated (from here we can go):<br>
    7.1 Text/content of the message<br>
    7.2 The thread time of the message<br>
    7.3 The thread count (reply count)<br>
    7.4 The number of user that reply the message (count of users that participated in the thread)<br>
    7.5 The time the last thread message was sent <br>
    7.6 The users that participated in the thread (their ids are stored as well)<br>

In [ ]:
def get_top_20_user(data, channel='Random'):
    """get user with the highest number of message sent to any channel"""

    data['sender_name'].value_counts()[:20].plot.bar(figsize=(15, 7.5))
    plt.title(f'Top 20 Message Senders in #{channel} channels', size=15, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=14);
    plt.xticks(size=12); plt.yticks(size=12);
    plt.show()

    data['sender_name'].value_counts()[-10:].plot.bar(figsize=(15, 7.5))
    plt.title(f'Bottom 10 Message Senders in #{channel} channels', size=15, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=14);
    plt.xticks(size=12); plt.yticks(size=12);
    plt.show()

def draw_avg_reply_count(data, channel='Random'):
    """who commands many reply?"""

    data.groupby('sender_name')['reply_count'].mean().sort_values(ascending=False)[:20]\
        .plot(kind='bar', figsize=(15,7.5));
    plt.title(f'Average Number of reply count per Sender in #{channel}', size=20, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

def draw_avg_reply_users_count(data, channel='Random'):
    """who commands many user reply?"""

    data.groupby('sender_name')['reply_users_count'].mean().sort_values(ascending=False)[:20].plot(kind='bar',
     figsize=(15,7.5));
    plt.title(f'Average Number of reply user count per Sender in #{channel}', size=20, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

def draw_wordcloud(msg_content, week):    
    # word cloud visualization
    allWords = ' '.join([twts for twts in msg_content])
    wordCloud = WordCloud(background_color='#975429', width=500, height=300, random_state=21, max_words=500, mode='RGBA',
                            max_font_size=140, stopwords=stopwords.words('english')).generate(allWords)
    plt.figure(figsize=(15, 7.5))
    plt.imshow(wordCloud, interpolation="bilinear")
    plt.axis('off')
    plt.tight_layout()
    plt.title(f'WordCloud for {week}', size=30)
    plt.show()

def draw_user_reaction(data, channel='General'):
    data.groupby('sender_name')[['reply_count', 'reply_users_count']].sum()\
        .sort_values(by='reply_count',ascending=False)[:10].plot(kind='bar', figsize=(15, 7.5))
    plt.title(f'User with the most reaction in #{channel}', size=25);
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()



In [ ]:
import os, sys
import re
import json
import glob
import datetime
from collections import Counter

# Get the current working directory
current_dir = os.getcwd()

# Get the parent directory (one level up)
parent_dir = os.path.dirname(current_dir)

# Construct the path to the 'network-analysis' directory
network_analysis_path = os.path.join(parent_dir, 'network-analysis')

# Append the path to sys.path
sys.path.append(network_analysis_path)


import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from wordcloud import WordCloud

from src.loader import *
from src.utils import *
from src.config import *

loaded_data = SlackDataLoader("anonymized")

usage: ipykernel_launcher.py [-h] [--output OUTPUT] --path PATH
                             [--channel CHANNEL] [--userfile USERFILE]
ipykernel_launcher.py: error: the following arguments are required: --path


SystemExit: 2

d:\10academy\week 0\network-analysis\venv3\lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Insight Extraction

Below are some useful questions to answer. Feel free to explore to answer other interesting questions that may be of help to get insight about student's behaviour, need, and future performance 

In [ ]:
# which user has the highest number of reply counts?

In [ ]:
# Visualize reply counts per user per channel

In [ ]:
# what is the time range of the day that most messages are sent?


In [ ]:
# what kind of messages are replied faster than others?

In [ ]:
# Relationship between # of messages and # of reactions

In [ ]:
# Classify messages into different categories such as questions, answers, comments, etc.

In [ ]:
# Which users got the most reactions?

In [ ]:
# Model topics mentioned in the channel

In [ ]:
# What are the topics that got the most reactions?

### Harder questions to look into

In [ ]:
# Based on messages, reactions, references shared, and other relevant data such as classification of questions into techical question, comment, answer, aorder stu the python, statistics, and sql skill level of a user?